In [ ]:
import os
import subprocess
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Docker container name and paths
docker_container_name = "4f97a78da6c07f8f6560c34d5914fa58da69384fbb2c10c14f0a4e7f9ad158e5"
container_path = "/mnt/extra-addons/custom_gmail"
local_path = "C:/Users/admin/Desktop/odoo18_dev/custom/custom_gmail"

# Ensure the local folder exists
if not os.path.exists(local_path):
    os.makedirs(local_path)

# Function to check and create directory in Docker container if it doesn't exist
def create_directory_in_container(directory_in_container):
    # Check if directory exists in the container
    check_command = [
        "docker", "exec", docker_container_name, "bash", "-c", f"test -d {directory_in_container} || echo 'missing'"
    ]
    result = subprocess.run(check_command, capture_output=True, text=True)

    if 'missing' in result.stdout:
        print(f"Directory {directory_in_container} is missing in the container, creating it...")
        # Create the directory if it's missing
        create_command = [
            "docker", "exec", docker_container_name, "bash", "-c", f"mkdir -p {directory_in_container}"
        ]
        create_result = subprocess.run(create_command, capture_output=True, text=True)
        if create_result.returncode == 0:
            print(f"Successfully created directory: {directory_in_container}")
        else:
            print(f"Failed to create directory. Error: {create_result.stderr}")
    else:
        print(f"Directory {directory_in_container} exists in the container.")

# Function to sync a single file to the Docker container
def sync_file_to_container(src_path, relative_path):
    # Chuyển đổi đường dẫn Windows thành dạng Unix để tương thích với Docker
    dest_path = os.path.join(container_path, relative_path).replace("\\", "/")
    dest_dir = os.path.dirname(dest_path)  # Extract the directory part of the destination path

    # Ensure the target directory exists in the Docker container
    create_directory_in_container(dest_dir)

    print(f"Syncing {src_path} to {docker_container_name}:{dest_path}...")
    
    # Sử dụng docker cp để đồng bộ tệp đã thay đổi
    sync_command = [
        "docker", "cp", src_path, f"{docker_container_name}:{dest_path}"
    ]
    result = subprocess.run(sync_command, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"Successfully synced {src_path} to Docker container.")
    else:
        print(f"Failed to sync {src_path}. Error: {result.stderr}")

# Function to download all files from Docker container to local folder (first run)
def download_from_container():
    print(f"Downloading all files from {docker_container_name}:{container_path} to {local_path}...")
    download_command = [
        "docker", "cp", f"{docker_container_name}:{container_path}.", local_path
    ]
    result = subprocess.run(download_command, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"Successfully downloaded files to {local_path}.")
    else:
        print(f"Failed to download files from container. Error: {result.stderr}")

# File change handler for monitoring the local folder
class LocalFileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        # Ignore directory modifications and temporary files
        if event.is_directory or event.src_path.endswith('~'):
            return
        
        # Calculate the relative path of the changed file
        relative_path = os.path.relpath(event.src_path, local_path)
        sync_file_to_container(event.src_path, relative_path)

    def on_created(self, event):
        # Sync new files to the Docker container
        if not event.is_directory:
            relative_path = os.path.relpath(event.src_path, local_path)
            sync_file_to_container(event.src_path, relative_path)

# Function to start monitoring the local folder for changes
def start_monitoring():
    event_handler = LocalFileChangeHandler()
    observer = Observer()
    observer.schedule(event_handler, path=local_path, recursive=True)
    observer.start()
    print(f"Monitoring changes in {local_path}...")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Function to check if this is the first run (local folder is empty)
def is_first_run():
    return not os.listdir(local_path)

def main():
    if is_first_run():
        # First run: download everything from the Docker container to the local folder
        download_from_container()
    else:
        # Start monitoring the folder for changes and sync to Docker container
        start_monitoring()

# Corrected main check
if __name__ == "__main__":
    main()


Monitoring changes in C:/Users/admin/Desktop/odoo18_dev/custom/custom_gmail...
Directory /mnt/extra-addons/custom_gmail/models is missing in the container, creating it...
Failed to create directory. Error: mkdir: cannot create directory '/mnt/extra-addons/custom_gmail': No such file or directory

Syncing C:/Users/admin/Desktop/odoo18_dev/custom/custom_gmail\models\gmail_config.py to 4f97a78da6c07f8f6560c34d5914fa58da69384fbb2c10c14f0a4e7f9ad158e5:/mnt/extra-addons/custom_gmail/models/gmail_config.py...
Failed to sync C:/Users/admin/Desktop/odoo18_dev/custom/custom_gmail\models\gmail_config.py. Error: Error response from daemon: error while creating mount source path '/run/desktop/mnt/host/c/Users/admin/Desktop/odoo18_dev/custom/config': mkdir /run/desktop/mnt/host/c/Users/admin/Desktop/odoo18_dev/custom/config: file exists

Directory /mnt/extra-addons/custom_gmail/models is missing in the container, creating it...
Failed to create directory. Error: mkdir: cannot create directory '/mnt/